In [ ]:
!pip install bitsandbytes transformers accelerate compel -q

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!wget https://raw.githubusercontent.com/kazunator/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py

--2023-12-09 21:32:17--  https://raw.githubusercontent.com/kazunator/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78414 (77K) [text/plain]
Saving to: ‘train_dreambooth_lora_sdxl.py’

train_dreambooth_lo 100%[===================>]  76.58K  --.-KB/s    in 0.002s  

2023-12-09 21:32:17 (36.1 MB/s) - ‘train_dreambooth_lora_sdxl.py’ saved [78414/78414]



In [ ]:
!wget https://raw.githubusercontent.com/A-cunminF/diffusers_utils/main/convert_diffusers_lora_bin_to_webui.py

--2023-12-09 22:12:03--  https://raw.githubusercontent.com/A-cunminF/diffusers_utils/main/convert_diffusers_lora_bin_to_webui.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4319 (4.2K) [text/plain]
Saving to: ‘convert_diffusers_lora_bin_to_webui.py’

convert_diffusers_l 100%[===================>]   4.22K  --.-KB/s    in 0s      

2023-12-09 22:12:03 (71.3 MB/s) - ‘convert_diffusers_lora_bin_to_webui.py’ saved [4319/4319]



In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


In [ ]:
import time
import random
import requests
from PIL import Image
from io import BytesIO
from datasets import load_dataset, Dataset, DatasetDict
dataset = load_dataset("SilentAntagonist/dalle3-reddit-cogvlm-captions")

# New dataset lists
images = []
captions = []
resolutions = []

def download_image(url, max_retries=5):
    retries = 0
    backoff_factor = 1  # Initial wait time between retries in seconds
    max_backoff = 5  # Maximum wait time
    while retries < max_retries:
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
            img = Image.open(BytesIO(response.content))
            return img
        except requests.exceptions.RequestException as e:
            wait_time = backoff_factor * (2 ** retries)
            if wait_time > max_backoff:
                print(f"Reached maximum backoff time, skipping: {url}")
                return None
            print(f"Failed to download {url}: {e}, retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            retries += 1
    print(f"Failed to download {url} after {max_retries} retries, skipping.")
    return None

# Process dataset
for item in dataset['train']:
    img = download_image(item['image_url'])
    if img:
        images.append(img)
        # 95% chance to choose long caption
        caption = item['long_caption'] if random.random() < 0.95 else item['short_caption']
        captions.append(caption)
        resolutions.append(item['resolution'])

# Create new dataset
new_dataset = Dataset.from_dict({
    'image': images,
    'caption': captions,
    'resolution': resolutions
})

# Convert to DatasetDict if needed
dataset_dict = DatasetDict({
    'train': new_dataset
})

# Save dataset to disk (Colab environment) before uploading
dataset_dict.save_to_disk('/content/my_new_dataset')



Failed to download https://i.redd.it/n6xjkflqbc3c1.jpg: 404 Client Error: Not Found for url: https://i.redd.it/n6xjkflqbc3c1.jpg, retrying in 1 seconds...
Failed to download https://i.redd.it/n6xjkflqbc3c1.jpg: 404 Client Error: Not Found for url: https://i.redd.it/n6xjkflqbc3c1.jpg, retrying in 2 seconds...
Failed to download https://i.redd.it/n6xjkflqbc3c1.jpg: 404 Client Error: Not Found for url: https://i.redd.it/n6xjkflqbc3c1.jpg, retrying in 4 seconds...
Reached maximum backoff time, skipping: https://i.redd.it/n6xjkflqbc3c1.jpg
Failed to download https://i.redd.it/f0cqygoqbc3c1.jpg: 404 Client Error: Not Found for url: https://i.redd.it/f0cqygoqbc3c1.jpg, retrying in 1 seconds...
Failed to download https://i.redd.it/f0cqygoqbc3c1.jpg: 404 Client Error: Not Found for url: https://i.redd.it/f0cqygoqbc3c1.jpg, retrying in 2 seconds...
Failed to download https://i.redd.it/f0cqygoqbc3c1.jpg: 404 Client Error: Not Found for url: https://i.redd.it/f0cqygoqbc3c1.jpg, retrying in 4 secon

Saving the dataset (0/3 shards):   0%|          | 0/4847 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
dataset_dict.push_to_hub("dalleTestData")

NameError: ignored

In [ ]:
import gc
import torch
from huggingface_hub import whoami, notebook_login, upload_folder, create_repo
from train_dreambooth_lora_sdxl import save_model_card


!accelerate config default

# Define the parameters for the training script
local_dir = "./dataset_images/"
output_dir = "custom_dog_model"





accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --dataset_name="Yorth/dalleTestDataFiltered2" \
  --output_dir="{output_dir}" \
  --caption_column="caption" \
  --instance_prompt="a photo of TOK dog" \
  --mixed_precision="fp16" \
  --resolution=1024 \
  --train_batch_size=4 \
  --gradient_accumulation_steps=3 \
  --gradient_checkpointing \
  --learning_rate=1e-4 \
  --snr_gamma=5.0 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --use_8bit_adam \
  --num_train_epochs=5 \
  --checkpointing_steps=717 \
  --train_text_encoder \
  --seed=0


2023-12-07 22:37:42.181631: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 22:37:42.181687: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 22:37:42.181728: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 22:37:43.322591: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
12/07/2023 22:37:44 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'feature_extractor', 'image_encoder'} was not found in config. Values 

In [ ]:
from diffusers import StableDiffusionPipeline
from compel import Compel

pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
compel = Compel(tokenizer=pipeline.tokenizer, text_encoder=pipeline.text_encoder)

# upweight "ball"
prompt = "a cat playing with a ball++ in the forest"
conditioning = compel.build_conditioning_tensor(prompt)
conditioning = compel(prompt)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [ ]:
# After training, generate a model card and upload the trained model to the Hugging Face Hub

repo_id = f"Yorth/dalletest5ep"
repo_id = create_repo(repo_id, exist_ok=True).repo_id

# Save the model card to the repository
save_model_card(
    repo_id=repo_id,
    images=[],  # You can add paths to sample images here
    base_model="stabilityai/stable-diffusion-xl-base-1.0",
    train_text_encoder=True,
    validation_prompt=None,
    repo_folder=output_dir,
    vae_path="madebyollin/sdxl-vae-fp16-fix",
)

# Upload the trained model to the Hugging Face Hub
upload_folder(
    repo_id=repo_id,
    folder_path=output_dir,
    commit_message="End of training",
    ignore_patterns=["step_*", "epoch_*"],
)

# Clean up to free memory
del gc.garbage[:]
torch.cuda.empty_cache()

events.out.tfevents.1701981936.51d491a54df6.2895.1:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1701981936.51d491a54df6.2895.0:   0%|          | 0.00/26.2k [00:00<?, ?B/s]

In [ ]:
from huggingface_hub.repocard import RepoCard
from diffusers import DiffusionPipeline
import torch




pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.load_lora_weights("Yorth/textencoderLora1")
image = pipe("Two animated characters, a male and a female, are surrounded by blue electric currents amidst a night setting. The man, on the left, appears to be reaching out to the woman, who is on the right, with a focused expression. The woman's attire and accessories suggest a mystical theme. There is a fox clinging onto the leg of the man.", num_inference_steps=40).images[0]
image.save("sks_dog.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch


generator = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    safety_checker=None
)
generator.load_lora_weights("Yorth/textencoderLora1")

generator.unet.load_attn_procs("Yorth/textencoderLora1")
generator.unet.save_attn_procs('./')

Keyword arguments {'safety_checker': None} are not expected by StableDiffusionXLPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

AttributeError: ignored

In [ ]:
import safetensors
import torch

pt_state_dict = safetensors.torch.load_file(
    "/content/pytorch_lora_weights.safetensors", device="cpu"
)
torch.save(pt_state_dict, "pt_state_dict.bin")
# OK

In [ ]:
!python convert_diffusers_lora_bin_to_webui.py --lora_bin_path=/content/pt_state_dict.bin --output_path=/content/pytorch_lora_weights2.safetensors